In [1]:
from PL_MMD_AAE import *
import json
exp_name = 'MMD_AAE/v1'
exp_name = 'resnet/resnet18_v1'
f = open(f'/root/device_fingerprint/configs/'+ exp_name + '.json')
        # f = open(f'/root/autoencoder_denoiser/configs_baseline_selection/'+ name + '.json')
        # global config
        
config = json.load(f)
model = MMD_AAE(config)

input = torch.rand([16,2,256])

In [2]:
# model.encoder.avgpool = nn.AdaptiveAvgPool1d(output_size=1)

In [3]:
logit, output_feature = model(input, feat = True)
output_feature.shape

torch.Size([16, 512, 8])

In [4]:
from  model_factory import *
# up1 = Up(in_channels=512, out_channels= 256, linear= True)
# f1 = up1(output_feature)
decoder = Decoder(256, 2, True)

In [5]:
f1 = decoder(output_feature)
f1.shape

torch.Size([16, 2, 256])

In [6]:
f2 = up1.conv.double_conv(f1)

NameError: name 'up1' is not defined

In [ ]:
f2.shape

In [ ]:
up1